In [1]:
import yapo as y
import pandas as pd

## Asset Creation

The library can create an `asset` in several ways. The method `portfolio_asset` accepts parameters:

- either `name`, or `names` of asset(s) to be created
- optional `start/end_period` to define date range. If no is provided then the library takes maximal range 
- optional `currency` to compute further activities. If no is provided then the library takes the default value for the financial symbol

In [2]:
y.portfolio_asset(name='micex/SBER')

PortfolioAsset(
     symbol: micex/SBER,
     currency: Currency(RUB),
     period_min: 2011-11,
     period_max: 2019-04
)

In [3]:
y.portfolio_asset(name='micex/SBER', start_period='2010-1')

PortfolioAsset(
     symbol: micex/SBER,
     currency: Currency(RUB),
     period_min: 2011-11,
     period_max: 2019-04
)

In [4]:
y.portfolio_asset(name='micex/SBER', start_period='2010-1', end_period='2013-1')

PortfolioAsset(
     symbol: micex/SBER,
     currency: Currency(RUB),
     period_min: 2011-11,
     period_max: 2013-01
)

In [5]:
asset=y.portfolio_asset(name='micex/SBER', 
                        start_period='2010-1', end_period='2013-1', 
                        currency='usd')
asset

PortfolioAsset(
     symbol: micex/SBER,
     currency: Currency(USD),
     period_min: 2011-11,
     period_max: 2013-01
)

## Basic Activities

Every asset has the `close` values. The `close` method returns `TimeSeries` object that wraps `numpy` array with additional meta-information: `start_period`, `end_period`, and `TimeSeriesKind`

In [6]:
close = asset.close()
close

TimeSeries(start_period=2011-11, end_period=2013-01, kind=TimeSeriesKind.VALUES, values=[2.79519565 2.4661372  2.96956663 3.46075861 3.23442966 3.19963764
 2.51734158 2.63248509 2.78767619 2.88479999 2.94305056 2.91163894
 2.94334519 3.05998479 3.64963017]

In [7]:
close.values

array([2.79519565, 2.4661372 , 2.96956663, 3.46075861, 3.23442966,
       3.19963764, 2.51734158, 2.63248509, 2.78767619, 2.88479999,
       2.94305056, 2.91163894, 2.94334519, 3.05998479, 3.64963017])

In [8]:
close.start_period, close.end_period

(Period('2011-11', 'M'), Period('2013-01', 'M'))

Next thing we derive from the `Close` values is [the `Return`](https://okama.io/#/glossary?page=cumulative-return). The `return` produces `TimeSeries` object that has one-month shorter `start_period` and different `kind`

In [9]:
asset.rate_of_return()

TimeSeries(start_period=2011-12, end_period=2013-01, kind=TimeSeriesKind.DIFF, values=[-0.11772287  0.20413683  0.16540864 -0.06539865 -0.01075677 -0.21324166
  0.04574012  0.05895232  0.03484042  0.02019224 -0.01067315  0.01088949
  0.03962824  0.19269553]